# AI-Powered Micro-Activity Recommendation System  
### Using Reinforcement Learning and Hybrid AI

**Student Name:**  
**Project Track:** Recommendation Systems / Applied ML  
**Mentor:**  

This notebook presents the design, implementation, and evaluation of a personalized micro-activity recommendation system that learns from user feedback using reinforcement learning.

## 1. Problem Definition & Objective

### a. Selected Project Track
This project falls under the **Recommendation Systems / Applied Machine Learning** track.

### b. Problem Statement
Users often waste short free time slots due to decision fatigue and lack of personalized suggestions. Existing productivity tools provide static or generic recommendations that do not adapt to individual preferences.

This project aims to build an AI system that recommends short, context-aware activities and continuously improves its suggestions using user feedback.

### c. Real-World Relevance & Motivation
Micro-moments (5–30 minutes) are common in daily life, especially for students and professionals. Efficient use of these moments can improve productivity, mental well-being, and habit formation.

## 2. Data Understanding & Preparation

### a. Dataset Source
The dataset used in this project is a **custom curated synthetic dataset** of micro-activities.
It was manually designed to simulate real-world recommendation scenarios.

Each activity contains:
- Activity name
- Category
- Suitable context (energy, location, duration)
- Description

In [ ]:
import json
import pandas as pd

with open("activities_dataset.json", "r") as f:
    data = json.load(f)

df = pd.DataFrame(data)
df.head()

### b. Data Exploration
The dataset consists of ~50–60 activities across multiple categories such as:
- Physical
- Mental
- Creative
- Relaxation
- Productivity

This diversity allows the recommendation system to adapt to different user contexts.

In [ ]:
df.info()
df['tags'].value_counts()

### c. Data Cleaning & Feature Engineering
- Categories and context tags were normalized
- No missing values were present
- Contextual attributes were converted into filters used during recommendation

### d. Handling Missing Values or Noise
Since the dataset is synthetic and curated, no missing values or noisy entries were found.

## 3. Model / System Design

### a. AI Technique Used
- Recommendation System
- Reinforcement Learning (Multi-Armed Bandit)
- Hybrid AI (ML + LLM)

### b. System Architecture
1. User provides context (time, energy, location)
2. Activities are filtered from dataset
3. Reinforcement learning ranks activities
4. Top recommendations are shown
5. User feedback updates the learning model

### c. Justification of Design Choices
- **Reinforcement Learning** allows the system to learn from user feedback instead of static rules
- **Thompson Sampling** balances exploration and exploitation efficiently
- **Hybrid LLM usage** prevents cold-start issues and adds creativity
- **SQLite storage** enables persistent learning across sessions

## 4. Core Implementation

In [ ]:
import random
import numpy as np
import sqlite3
from collections import defaultdict

# Simplified Bandit Agent with SQL persistence
class BanditAgent:
    def __init__(self, activities, db_file="feedback.db"):
        self.activities = activities
        self.db_file = db_file
        self.init_db()
        self.success = self.load_params('alpha')
        self.failure = self.load_params('beta')

    def init_db(self):
        conn = sqlite3.connect(self.db_file)
        c = conn.cursor()
        c.execute('''CREATE TABLE IF NOT EXISTS beta_params (
                        activity TEXT PRIMARY KEY,
                        alpha REAL DEFAULT 1,
                        beta REAL DEFAULT 1
                    )''')
        conn.commit()
        conn.close()

    def load_params(self, param):
        conn = sqlite3.connect(self.db_file)
        c = conn.cursor()
        c.execute(f"SELECT activity, {param} FROM beta_params")
        rows = c.fetchall()
        conn.close()
        params = defaultdict(lambda: 1)
        for activity, value in rows:
            params[activity] = value
        return params

    def save_params(self):
        conn = sqlite3.connect(self.db_file)
        c = conn.cursor()
        for act in self.activities:
            c.execute("INSERT OR REPLACE INTO beta_params (activity, alpha, beta) VALUES (?, ?, ?)",
                      (act, self.success[act], self.failure[act]))
        conn.commit()
        conn.close()

    def recommend(self):
        sampled_scores = {
            a: np.random.beta(self.success[a], self.failure[a])
            for a in self.activities
        }
        return max(sampled_scores, key=sampled_scores.get)

    def update(self, activity, reward):
        if reward == 1:
            self.success[activity] += 1
        else:
            self.failure[activity] += 1
        self.save_params()

The system treats each activity as an "arm" in a multi-armed bandit.
User feedback acts as the reward signal, allowing the model to improve recommendations over time.
The SQLite database persists the beta parameters for learning across sessions.

## 5. Evaluation & Analysis

### a. Evaluation Metrics
Since this is an interactive recommendation system:
- Qualitative evaluation was used
- User satisfaction based on feedback trends

In [ ]:
agent = BanditAgent(df['name'].tolist())

for _ in range(10):
    rec = agent.recommend()
    reward = random.choice([0, 1])  # simulated feedback
    agent.update(rec, reward)
    print(f"Recommended: {rec}, Reward: {reward}")

### b. Performance Analysis
- Recommendations become more personalized after multiple interactions
- Repeated disliked activities are gradually avoided

### c. Limitations
- Requires user interaction to learn
- Single-user focus

## 6. Ethical Considerations & Responsible AI

- No personal or sensitive data is collected
- Dataset avoids harmful or unsafe activities
- Feedback-based learning reduces biased assumptions
- LLM usage is controlled and disclosed